### Imports

In [439]:
import pandas as pd
from pycaret.regression import *
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
import pickle
from datetime import timedelta
import talib
import numpy as np

In [440]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Dataset

In [441]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

columns_to_model = [
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    # 'SMA_20',
    # 'EMA_20',
    # 'Upper_Band',
    # 'Middle_Band',
    # 'Lower_Band',
    # 'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

In [442]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])

display(dataset.head())

,Open_time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,...,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
0,2021-09-27,28.84,30.26,27.50,27.71,11094418.10,320340017.82,449178.00,43160.90,44350.00,...,1.00,0.00,0.00,1.00,0.00,81.00,525.00,364.00,270000.00,Lateral
1,2021-09-28,27.72,28.38,26.14,26.31,9784154.53,267560973.69,362304.00,42147.35,42787.38,...,2.00,1.00,1.00,0.00,0.02,87.00,472.00,331.00,204000.00,Bajista
2,2021-09-29,26.31,28.59,26.11,27.28,10207287.33,280535376.38,376232.00,41025.01,42590.00,...,0.00,0.00,0.00,0.00,0.00,64.00,594.00,495.00,216000.00,Alcista
3,2021-09-30,27.28,28.99,27.13,28.62,8201551.44,230622998.43,339737.00,41524.29,44141.37,...,2.00,1.00,0.00,1.00,0.75,77.00,419.00,464.00,202000.00,Alcista
4,2021-10-01,28.61,32.33,28.50,31.94,13176348.21,402700970.65,735059.00,43820.01,48495.00,...,1.00,1.00,0.00,0.00,0.82,71.00,477.00,664.00,492000.00,Alcista


In [443]:
dates = dataset['Open_time'][:-5]

dataset = dataset.round(2)

feature_dataset = dataset[columns]
# feature_dataset['Open_time'] = dataset['Open_time']
# feature_dataset.set_index('Open_time', inplace=True)

validation = feature_dataset[-5:]
feature_dataset = feature_dataset[:-5]

n_days_to_predict = 5

In [444]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,...,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,2021-09-27,28.84,30.26,27.50,27.71,449178.00,42147.35,39776.84,1001487.00,2925.59,...,0.00,0.00,0.00,139,135,1.00,81.00,525.00,364.00,270000.00
1,2021-09-28,27.72,28.38,26.14,26.31,362304.00,41026.54,43372.26,1045389.00,2804.91,...,4.00,2.00,5273.00,93,122,2.00,87.00,472.00,331.00,204000.00
2,2021-09-29,26.31,28.59,26.11,27.28,376232.00,41524.28,33511.53,884909.00,2850.45,...,22.00,40.00,54144.00,112,145,0.00,64.00,594.00,495.00,216000.00
3,2021-09-30,27.28,28.99,27.13,28.62,339737.00,43824.10,46381.23,1197815.00,3000.61,...,15.00,23.00,39220.00,116,147,2.00,77.00,419.00,464.00,202000.00
4,2021-10-01,28.61,32.33,28.50,31.94,735059.00,48141.61,66244.87,1771237.00,3309.91,...,24.00,35.00,63183.00,171,141,1.00,71.00,477.00,664.00,492000.00


(903, 31)


,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,...,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
903,2024-03-18,10.08,10.46,9.60,9.90,245319.00,67609.99,55691.08,2464515.00,3520.46,...,34.00,43.00,84706.00,696,471,0.00,43.00,343.00,228.00,154000.00
904,2024-03-19,9.90,9.99,8.60,8.77,341363.00,61937.40,101005.32,3593832.00,3158.64,...,120.00,126.00,135180.00,961,509,1.00,56.00,534.00,433.00,221000.00
905,2024-03-20,8.77,9.57,8.49,9.48,267797.00,67840.51,90420.59,3549793.00,3516.53,...,185.00,117.00,112997.00,866,555,1.00,40.00,473.00,386.00,171000.00
906,2024-03-21,9.48,9.58,9.07,9.18,156774.00,65501.27,53357.48,2388390.00,3492.85,...,64.00,81.00,66543.00,692,533,0.00,24.00,350.00,290.00,101000.00
907,2024-03-22,9.18,9.37,8.69,8.94,147578.00,63796.64,51482.38,2492881.00,3336.35,...,57.00,66.00,68616.00,681,546,0.00,41.00,252.00,206.00,92000.00


(5, 31)

### Model

#### Generacion de los modelos

In [445]:
# train_size = 0.8
# train_data, test_data = train_test_split(feature_dataset, train_size=train_size, shuffle=False)

In [446]:
h2o.init()

selected_models = {}

for column in columns_to_model:
    
    # A cada registro, le asigno como target el valor de Column del dia siguiente,
    # de esta forma, los N modelos son entrenados para predecir el valor de los N
    # atributos en un dia siguiente.
    training_data = feature_dataset
    training_data['Next_Day_Target'] = feature_dataset[column].shift(-1)
    training_data = training_data[:-1] #Tengo que sacar el ultimo registro porque no tendra valor del target
    ##
    h2o_df = h2o.H2OFrame(training_data.drop(columns=["Open_time"]))

    aml = H2OAutoML(max_models=25, seed=1234) #TODO: SACAR EL max_models
    aml.train(y='Next_Day_Target', training_frame=h2o_df)

    leaderboard = aml.leaderboard
    print(leaderboard)

    top_models = leaderboard.head(3)["model_id"].as_data_frame()["model_id"].tolist()

    best_model_id = top_models[0]
    best_model = h2o.get_model(best_model_id)
    
    selected_models[column] = best_model

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 hours 7 mins
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_mmarchetta_dfdxaj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.472 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                       rmse        mse       mae       rmsle    mean_residual_deviance
GLM_1_AutoML_247_20240422_150648                           0.16438   0.0270209  0.102224  0.0108438                  0.0270209
StackedEnsemble_BestOfFamily_1_AutoML_247_20240422_150648  0.171651  0.0294642  0.100749  0.00983934                 0.0294642
StackedEnsemble_AllModels_1_AutoML_247_20240422_150648     0.172071  0.0296084  0.095001  0.00901993                 0.0296084
GBM_5_AutoML_247_20240422_150648                           0.268549  0.0721187  0.105607  0.00947209                 0.0721187
GBM_4_AutoML_247_20240422_150648                           0.273752  0.0749403  0.108731  0.00965064                 0.0749403
GBM_grid_1_AutoML_247_20240422_150648_model_4   

In [447]:
predictions_dict = {}

for column, model in selected_models.items():
    predictions = model.predict(h2o.H2OFrame(validation.drop(columns=["Open_time"])))
    predictions_dict[column] = predictions

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |██████████████████████████████████████████████████

In [448]:
for column, predictions in predictions_dict.items():
    print(f"Predicciones para los próximos días de {column}:")
    print(predictions)

Predicciones para los próximos días de Open:
  predict
  9.9575
  8.94215
  9.32918
  9.28929
  9.01623
[5 rows x 1 column]

Predicciones para los próximos días de High:
  predict
 10.0663
  9.14965
  9.32176
  9.01123
  8.79655
[5 rows x 1 column]

Predicciones para los próximos días de Low:
  predict
  9.16624
  7.79157
  8.32764
  8.50675
  8.22469
[5 rows x 1 column]

Predicciones para los próximos días de Close:
  predict
  9.96685
  9.22537
  9.27956
  9.39607
  9.07004
[5 rows x 1 column]

Predicciones para los próximos días de Number of trades:
  predict
   208702
   226976
   238972
   228411
   203376
[5 rows x 1 column]

Predicciones para los próximos días de Close_BTCUSDT:
  predict
  68772.4
  60049.9
  66506.8
  66238.5
  63756.6
[5 rows x 1 column]

Predicciones para los próximos días de Volume_BTCUSDT:
  predict
  48715.3
  59659.5
  60348.2
  48497.8
  47278
[5 rows x 1 column]

Predicciones para los próximos días de Number_of_trades_BTCUSDT:
    predict
2.18552e+06
2.

#### Guardo el mejor modelo para cada columna

In [449]:
for column, model in selected_models.items():
    file_name = f"modelo_{column}.h2o"
    h2o.save_model(model, 'h2o_models', filename=file_name)

#### Vuelvo a armar el mejor modelo para cada columna

In [450]:
loaded_models = {}
for column in columns_to_model:
    model_path = f"modelo_{column}.h2o"
    loaded_models[column] = h2o.load_model(f"h2o_models/{model_path}")

#### Prediccion 1 dia en el futuro

In [451]:
def hacer_predicciones_para_dia_siguiente(trained_models, last_day_data):
    """
    Predice los valores para el día siguiente utilizando los modelos entrenados y el último registro de datos.

    Args:
    - trained_models: Un diccionario donde las claves son las características y los valores son los modelos entrenados.
    - last_day_data: Pandas Series que representa el último registro de datos con todas las características excepto la variable objetivo.

    Returns:
    - Pandas DataFrame con un solo registro que contiene las predicciones para el día siguiente.
    """
    # Crear un diccionario para almacenar las predicciones
    predictions = {}

    # Hacer predicciones para cada característica utilizando los modelos entrenados
    for feature, model in trained_models.items():
        # Preparar los datos para la predicción (convertir a DataFrame de H2O)
        h2o_last_day_data = h2o.H2OFrame(last_day_data)
        
        # Realizar la predicción para la característica actual
        prediction = model.predict(h2o_last_day_data).as_data_frame().iloc[0, 0]
        
        # Almacenar la predicción en el diccionario
        predictions[feature] = prediction

    # Convertir el diccionario de predicciones en un DataFrame de pandas
    predictions_df = pd.DataFrame(predictions, index=[0])

    return predictions_df

In [452]:
predictions_next_day = hacer_predicciones_para_dia_siguiente(loaded_models, feature_dataset.drop(columns=["Open_time", "Next_Day_Target"])[-1:])

# Muestra el DataFrame con las predicciones para el día siguiente
display(predictions_next_day)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,...,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,10.02,10.04,9.25,9.99,233830.73,68243.94,52031.81,2307654.24,3675.65,596923.84,...,52.33,43.13,68704.25,595.77,313.21,0.65,37.79,392.95,341.34,154147.20


#### Prediccion de los N dias en el futuro

In [453]:
def calcular_indicadores_tecnicos(historico_precio):
    close_prices = np.array(historico_precio['Close'], dtype=np.double)
    # high_prices = np.array(historico_precio['High'], dtype=np.double)
    # low_prices = np.array(historico_precio['Low'], dtype=np.double)

    # Media Móvil Simple (SMA)
    sma_20 = talib.SMA(close_prices, timeperiod=20)

    # Media Móvil Exponencial (EMA)
    ema_20 = talib.EMA(close_prices, timeperiod=20)

    # Bandas de Bollinger
    upper_band, middle_band, lower_band = talib.BBANDS(close_prices, timeperiod=20, nbdevup=2, nbdevdn=2)

    # Índice de Fuerza Relativa (RSI)
    rsi = talib.RSI(close_prices, timeperiod=14)

    # Crear un DataFrame para almacenar los indicadores técnicos
    indicadores = pd.DataFrame({
        'Open_time': historico_precio['Open_time'],
        'SMA_20': sma_20,
        'EMA_20': ema_20,
        'Upper_Band': upper_band,
        'Middle_Band': middle_band,
        'Lower_Band': lower_band,
        'RSI': rsi
    }, index=historico_precio.index)

    return indicadores

In [454]:
cantidad_dias_futuro = 5

for i in range(cantidad_dias_futuro):
    predictions_next_day = hacer_predicciones_para_dia_siguiente(loaded_models, feature_dataset.drop(columns=["Open_time", "Next_Day_Target"])[-1:])
    
    ultima_fecha = feature_dataset["Open_time"].iloc[-1]
    nueva_fecha = ultima_fecha + timedelta(days=1)
    
    nuevo_registro = pd.DataFrame({"Open_time": [nueva_fecha]})

    for column, prediction in predictions_next_day.items():
        nuevo_registro[column] = prediction
        
    feature_dataset = pd.concat([feature_dataset, nuevo_registro], ignore_index=True)

    nuevos_indicadores = calcular_indicadores_tecnicos(feature_dataset)
    
    for column in nuevos_indicadores.columns:
        feature_dataset.loc[feature_dataset.index[-1], column] = nuevos_indicadores.loc[nuevos_indicadores.index[-1], column]
    
    # display(nuevos_indicadores)mm
    

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

In [455]:
display(feature_dataset.tail(10))

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,...,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Next_Day_Target
898,2024-03-13,10.74,11.46,10.64,11.37,276468.00,73072.41,52659.71,2501197.00,4004.79,...,81.00,92576.00,275.00,205.00,1.00,73.00,578.00,553.00,164000.00,327000.00
899,2024-03-14,11.37,11.89,10.68,11.56,536988.00,71388.94,71757.63,2994869.00,3881.70,...,133.00,145727.00,211.00,181.00,4.00,29.00,754.00,677.00,327000.00,360000.00
900,2024-03-15,11.56,11.71,9.98,10.81,557152.00,69499.85,103334.04,3904445.00,3742.19,...,83.00,147460.00,238.00,106.00,0.00,25.00,493.00,430.00,360000.00,209000.00
901,2024-03-16,10.81,10.90,9.50,9.68,330505.00,65300.63,55926.95,2729019.00,3523.09,...,49.00,88095.00,670.00,471.00,0.00,20.00,513.00,403.00,209000.00,150000.00
902,2024-03-17,9.68,10.25,9.19,10.08,229683.00,68393.48,49742.22,2449156.00,3644.71,...,48.00,71390.00,693.00,413.00,0.00,21.00,295.00,277.00,150000.00,NaN
903,2024-03-18,10.02,10.04,9.25,9.99,233830.73,68243.94,52031.81,2307654.24,3675.65,...,43.13,68704.25,595.77,313.21,0.65,37.79,392.95,341.34,154147.20,NaN
904,2024-03-19,9.89,10.09,9.22,10.10,241173.30,68125.38,50698.63,2078555.34,3699.97,...,44.22,69391.89,489.66,254.96,0.59,47.52,441.57,398.45,161478.33,NaN
905,2024-03-20,9.96,10.26,9.34,10.15,251734.92,67868.90,49918.52,2028623.39,3695.27,...,48.71,69683.41,320.03,222.27,0.51,42.10,458.21,411.46,172678.90,NaN
906,2024-03-21,10.05,10.41,9.48,10.28,248340.58,67843.40,47125.40,2014721.14,3680.27,...,53.20,72057.40,325.43,200.76,0.41,41.48,473.37,435.32,171263.60,NaN
907,2024-03-22,10.19,10.59,9.59,10.34,255878.16,67681.27,48943.07,2078739.17,3678.90,...,55.18,74904.49,368.38,194.39,0.39,41.48,489.53,471.22,177933.54,NaN


Si predice maso menos bien para estas variables hacer lo siguiente:
- Evaluar los resultados por columna (definir alguna forma de hacer esto)

- Armar el clasificador (LSTM o CONV), entrenandolo sobre los datos del dataset original ( sin los ultimos 5 registros)
- Probar como predice el target (Tendencia) para los ultimos 5 registros con los datos generados por los modelos